In [3]:
import psycopg2
import json
from xml.etree import ElementTree as ET
import csv
import re

dbname = "dbt_poc"
user = "postgres"
password = "admin"
host = "localhost"
port = "5432"

def extract_and_rebuild_xml(xml_file, metadata_table, output_csv):
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

    cur = conn.cursor()
    cur.execute(f"SELECT tag FROM {metadata_table} WHERE active = TRUE")
    necessary_tags = [row[0] for row in cur.fetchall()]
    cur.close()
    conn.close()

    with open(xml_file, 'r', encoding='utf-8') as f:
        xml_content = f.read()

    root_tag = "genericSMF"
    pattern = rf'(<{root_tag}>.*?</{root_tag}>)'
    xml_documents = re.findall(pattern, xml_content, re.DOTALL)

    all_xml_str = []
    all_json_data = []

    for xml_doc in xml_documents:
        try:
            tree = ET.ElementTree(ET.fromstring(xml_doc))
            root = tree.getroot()

            def filter_tags(element):
                if element.tag in necessary_tags:
                    return element
                else:
                    return [child for child in element.iter() if child.tag in necessary_tags]

            filtered_elements = filter_tags(root)

            new_root = ET.Element(root.tag)
            for element in filtered_elements:
                new_root.append(element)

            xml_str = ET.tostring(new_root, encoding='unicode')
            json_data = json.dumps(xml_to_dict(xml_str), indent=4)
            all_xml_str.append(xml_str.strip())
            all_json_data.append(json_data.strip())
        except ET.ParseError as e:
            print(f"Error parsing XML document: {e}")
            continue

    with open(output_csv, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["attbr_xml", "attbr_json"])  
        for xml_str, json_data in zip(all_xml_str, all_json_data):
            writer.writerow([xml_str, json_data])

def xml_to_dict(xml_str):
    root = ET.fromstring(xml_str)
    return {root.tag: xml_to_dict_helper(root)}

def xml_to_dict_helper(element):
    if len(element) == 0:
        return element.text if element.text else ''
    result = {}
    for child in element:
        child_result = xml_to_dict_helper(child)
        if child.tag in result:
            if isinstance(result[child.tag], list):
                result[child.tag].append(child_result)
            else:
                result[child.tag] = [result[child.tag], child_result]
        else:
            result[child.tag] = child_result
    return result

xml_file = "C:\\Users\\asus\\Downloads\\madhu\\SMFsourcefile.xml"
metadata_table = "metadata.metadata_table"
output_csv = "E:\\project-upwork\\poc\\seeds\\xml_data.csv"

extract_and_rebuild_xml(xml_file, metadata_table, output_csv)
print(f"XML and JSON data saved to {output_csv}")


XML and JSON data saved to E:\project-upwork\poc\seeds\xml_data.csv
